https://deep-learning-study.tistory.com/686

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator
import spacy
import numpy as np
import random
import math
import time

ModuleNotFoundError: No module named 'torchtext.legacy'

In [5]:
seed=1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)#pytorch의 randomseed고정
torch.cuda.manual_seed(seed)#pytorch randomseed 고정
torch.backend.cudnn.deterministic=True#cudnn randomseed 를 고정

NameError: name 'random' is not defined

In [ ]:
import de_core_news_sm
import en_core_web_sm

spacy_en= en_core_web_sm.load()
spacy_de=de_core_news_sm.load()

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

SRC=Field(tokenize=tokenize_de, init_token='<sos>', eos_token='<sos>', lower=True)
TRG=Field(tokenize=tkenize_en, init_token'<sos>', eos_token='<sos>', lower=True)

In [ ]:
train_data, valid_data, test_data=Multi30k.splits(exts=('.de','.en'),fields=(SRC,TRG))
# train, validation, test 데이터를 불러오고, 다운로드 합니다.
# Multi30k dataset을 사용하여, 30,000개의 영어, 독일, 프랑스어 문장을 포함합니다.

In [ ]:
print(f'Number of training examples:{len(train_data.examples)}')
print(f'Number of validation examples:{len(valid_data.examples)}')
print(f'Number of testing examples:{len(test_data.examples)}')

In [ ]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data,min_freq=2)
#토큰화 전처리가 끝나면, 각단어에 고유한 정수를 맵핑해주는 정수 인코딩작업이 필요하다. 그리고 이 전처리를 위해서는 단어집합이 필요한데, 
#정의한 filed에 build_vocab을 이용하면 단어집합을 만들수 있다.
# min_freq=2는 2번 이상 등장한 토큰을 출력합니다.
# 토큰이 1번만 등장했다면 <unk>로 대체합니다.

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size=128
train_iterator, valid_iterator, test_iterator=BucketIterator.splits((train_data, valid_data, test_data),batch_size=batch_size,device=device)

In [3]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim=hid_dim
        self.n_layers=n_layers
        #임베딩:입력값을 emb_dim벡터로 변경
        self.embedding=nn.Embeding(input_dim, emb_dim)
        
        #embeding을 입력받아 hid_dim크기의 hidden state,cell출력
        self.rnn=nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        
        self.dropout=nn.Dropout(dropout)
    
    def forward(self,src):
        #src=[src.len,batch_size]
        embedded= self.dropout(self.embedding(src))
        #initial hidden state는 제로텐서
        outputs, (hidden,cell)=self.rnn(embedded)
        # output: [src_len, batch_size, hid dim * n directions]
        # hidden: [n layers * n directions, batch_size, hid dim]
        # cell: [n layers * n directions, batch_size, hid dim]
        return hidden, cell

NameError: name 'nn' is not defined